In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
import joblib

# Load dataset
df = pd.read_csv("Titanic.csv")

# Select and clean columns
df = df[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']].dropna()

# Encode categorical variable
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])  # male=1, female=0

# Features and target
X = df[['Pclass', 'Sex', 'Age', 'Fare']]
y = df['Survived']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build pipeline (scaling + classifier)
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(random_state=42))
])

# Grid search parameters
param_grid = {
    'clf__n_estimators': [50, 100],
    'clf__max_depth': [3, 5, None]
}

# Use KFold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=cv, scoring='accuracy')
grid.fit(X_train, y_train)

# Predict on test set
y_pred = grid.predict(X_test)

# Evaluation report
print("Best Parameters:", grid.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Save the best model
joblib.dump(grid.best_estimator_, 'best_titanic_model.pkl')
print("\n✅ Model saved to 'best_titanic_model.pkl'")


Best Parameters: {'clf__max_depth': 3, 'clf__n_estimators': 100}

Classification Report:
               precision    recall  f1-score   support

        Died       0.81      0.92      0.86       189
    Survived       0.67      0.42      0.52        73

    accuracy                           0.78       262
   macro avg       0.74      0.67      0.69       262
weighted avg       0.77      0.78      0.77       262


✅ Model saved to 'best_titanic_model.pkl'
